In [1]:
'''
MADE BY AAYUSH DESHMUKH
ON DATE 13-10-2021
A VIDEO CLASSSIFICATION MODEL THAT CAN CLASSIFY 100 DIFFERENT HUMAN ACTIVITIES
Activity_Classification.py
'''

'\nMADE BY AAYUSH DESHMUKH\nON DATE 13-10-2021\nA VIDEO CLASSSIFICATION MODEL THAT CAN CLASSIFY 100 DIFFERENT HUMAN ACTIVITIES\nActivity_Classification.py\n'

In [2]:
#Importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from keras_video import VideoFrameGenerator
import glob

In [3]:
#Taking a look at the distribution of the data
directory = "D:/AAYUSH/Deep Learning/Data_set/Activity recognition/UCF101/"
class_names = os.listdir(directory)

Data={}
for name in class_names:
    Data[name]=len(os.listdir(directory+'/'+name))

plt.figure(figsize = (150,100), )
plt.style.use("dark_background")
plt.bar(Data.keys(), Data.values())
plt.xlabel("Activities", fontsize=100)
plt.ylabel("Number of Images", fontsize=100)
plt.title("Number of images in every class", fontsize=100)
plt.xticks(range(len(class_names)), class_names, rotation=90, fontsize=100)
plt.yticks(fontsize=100)
plt.show()

In [4]:
#Defining hyperparameters
FRAME_SIZE = (320, 240)
FRAMES = 10
BATCH_SIZE = 64
#INPUT_SHAPE = (10, 320, 240, 3)
GLOB_PATTERN = str(directory+'{classname}/*.avi')
print(GLOB_PATTERN)

D:/AAYUSH/Deep Learning/Data_set/Activity recognition/UCF101/{classname}/*.avi


In [5]:
#Data(image) augmentation
DATA_AUG = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)

# Creating video frame generator
train_generator = VideoFrameGenerator(
    classes=class_names, 
    glob_pattern=GLOB_PATTERN,
    nb_frames=FRAMES,
    split_val=.2, 
    shuffle=True,
    #batch_size=BATCH_SIZE,
    #target_shape=FRAME_SIZE,
    nb_channel=3,
    transformation=DATA_AUG,
    use_frame_cache=True)

class ApplyEyeMakeup, validation count: 29, train count: 116
class ApplyLipstick, validation count: 22, train count: 92
class Archery, validation count: 29, train count: 116
class BabyCrawling, validation count: 26, train count: 106
class BalanceBeam, validation count: 21, train count: 87
class BandMarching, validation count: 31, train count: 124
class BaseballPitch, validation count: 30, train count: 120
class Basketball, validation count: 26, train count: 108
class BasketballDunk, validation count: 26, train count: 105
class BenchPress, validation count: 32, train count: 128
class Biking, validation count: 26, train count: 108
class Billiards, validation count: 30, train count: 120
class BlowDryHair, validation count: 26, train count: 105
class BlowingCandles, validation count: 21, train count: 88
class BodyWeightSquats, validation count: 22, train count: 90
class Bowling, validation count: 31, train count: 124
class BoxingPunchingBag, validation count: 32, train count: 131
class Box

In [6]:
valid_generator = train_generator.get_validation_generator()

Total data: 101 classes for 2630 files for validation


In [7]:
def activity_recognition_model():
    model=tf.keras.models.Sequential([
        tf.keras.layers.ConvLSTM2D(filters=16, kernel_size=(3,3), 
                                  activation='relu',input_shape=(10, 244, 244, 3), padding='same'),
       
        tf.keras.layers.Dropout(0.5),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1004, activation='relu'),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ])
    #opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)
    #opt= tf.keras.optimizers.SGD(learning_rate=0.005)
    
    #loss = tf.keras.losses.CategoricalCrossentropy()
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(loss=loss,
              optimizer=opt,
              metrics=['accuracy'])

    return model

In [8]:
model = activity_recognition_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 244, 244, 16)      11008     
_________________________________________________________________
dropout (Dropout)            (None, 244, 244, 16)      0         
_________________________________________________________________
flatten (Flatten)            (None, 952576)            0         
_________________________________________________________________
dense (Dense)                (None, 1004)              956387308 
_________________________________________________________________
dense_1 (Dense)              (None, 50)                50250     
_________________________________________________________________
dense_2 (Dense)              (None, 101)               5151      
Total params: 956,453,717
Trainable params: 956,453,717
Non-trainable params: 0
__________________________________________

In [9]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')>0.90):
            print("\nReached 90.0% accuracy so cancelling the training.")
            self.model.stop_training = True

callbacks = myCallback()


In [10]:
history = model.fit(train_generator,
                    steps_per_epoch=32,
                    validation_data=valid_generator,
                    verbose=1,
                    validation_steps=32,
                    callbacks=[callbacks])

InvalidArgumentError:  Specified a list with shape [?,244,244,3] from a tensor with shape [16,224,224,3]
	 [[node sequential/conv_lst_m2d/TensorArrayUnstack/TensorListFromTensor (defined at <ipython-input-10-44d8ac7f7a74>:6) ]] [Op:__inference_train_function_3683]

Function call stack:
train_function
